In [1]:
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout, Activation, Input, Conv2D, MaxPooling2D
from keras.models import Model
from keras import Sequential
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pickle
import os

In [2]:
def get_images(pasta):
    exts = ['.PNG','.JPG','.JPEG','.TIFF','.GIF','.BMP']
    tot_images = 0
    for sf in [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))]:
        subdir = os.path.join(pasta,sf)
        tot_images = tot_images + len([name for name in os.listdir(subdir) if os.path.splitext(name)[1].upper() in exts])
    return tot_images

In [3]:
# FUNÇÃO PARA LER O NOME DAS CLASSES
def get_labels(pasta):
    return [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))];


In [4]:
EPOCHS = 25;
BATCH_SIZE = 16;
IMG_SIZE = (224,224,3);

In [5]:
trainFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\train'
valFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\val'
labels = get_labels(trainFolder)
labels = np.array(labels);

# Organiza os labels em matriz e salva, para posterior uso em classificação
lb = LabelBinarizer();
labels = lb.fit_transform(labels);
f = open('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\budioes'+".pickle", "wb")
f.write(pickle.dumps(lb));
f.close();

ImageDataGenerator = Carrega aos poucos as imagens em memória para fazer a leitura (para não estourar a memória)

In [6]:
#ImageDataGenerator = objeto para buscar as imagens em uma pasta
#Treino
augTrain = ImageDataGenerator();
#Validação
augVal = ImageDataGenerator();

Relu = 

Maxpooling = selecionar o pixel com maior valor em uma dimensão 2D de 2x2 (no exemplo abaixo, poderia ser 3x3, 4x4 etc). Nesse caso, com uma janela 2x2, o maxpooling vai reduzir o tamanho da imagem pela metade.

In [7]:
model = Sequential()
#32 = camadas da imagem e 5x5 é o tamanho da janela de varredura
model.add(Conv2D(32, (5, 5), activation='relu'))
#maxpooling reduz o tamanho da imagem, 2x2 irá reduzir a imagem pela metade (vai pular 2 pixels a cada "janelamento")
model.add(MaxPooling2D((2, 2))) 
#Reduzir a janela de 5x5 para 3x3
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
#Transforma o "cubo" em 1 vetor e 1 dimensão (vai receber um cubo de 28x28x8 e transformar em um vetor de 6272)
model.add(Flatten())
#Camadas da rede neural:
#Vai receber um vetor de 6272 e usar de input nas camadas da rede
#Por isso temos que fazer várias camadas, para não "estrangular" a rede e obter o resultado final binário
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
#Resultado final da rede. No caso dos budiões, estamos trabalhando com 3 resultados possíveis. Por isso é 3.
model.add(Dense(3, activation='sigmoid'))

In [8]:
learning_rate = 0.01
optimizer = SGD(learning_rate=learning_rate)

In [9]:
#Reservando espaço de memória para a rede funcionar
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [10]:
#Função para buscar imagens
#TREINO
trainGen = augTrain.flow_from_directory(
    trainFolder, #caminho da imagem
    class_mode="categorical", #o nome da pasta onde está a imagem será o nome da classe
    target_size=(IMG_SIZE[0], IMG_SIZE[1]), #tamanho da imagem a ser redimensionada
    color_mode="rgb", #a imagem terá 3 canais RGB
    shuffle=True, #vai embaralhar as imagens enquanto faz a leitura
    batch_size=BATCH_SIZE); #de quantas em quantas imagens será feita a leitura (tamanho do BATCH)

Found 537 images belonging to 3 classes.


In [11]:
#Função para buscar imagens
#VALIDAÇÃO
valGen = augVal.flow_from_directory(
    valFolder,
    class_mode="categorical",
    target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE);

Found 135 images belonging to 3 classes.


In [12]:
#fit_generator vai de fato treinar a rede
trained_model = model.fit(trainGen, validation_data=valGen,
                        steps_per_epoch=get_images(trainFolder)//BATCH_SIZE,
                        validation_steps=get_images(valFolder) // BATCH_SIZE,
                        epochs = EPOCHS, verbose =1);
#Dividir o número de imagens pelo número de batchs para garantir que cada BATCH seja lido a cada época
#tanto no treino (steps_per_epoch) quanto na validação (validation_steps)
#verbose = dá output da rede a cada final de época


Epoch 1/25
33/33 [==============================] - 8s 173ms/step - loss: nan - accuracy: 0.2706 - val_loss: nan - val_accuracy: 0.2734
Epoch 2/25
33/33 [==============================] - 5s 151ms/step - loss: nan - accuracy: 0.2841 - val_loss: nan - val_accuracy: 0.2891
Epoch 3/25
33/33 [==============================] - 5s 151ms/step - loss: nan - accuracy: 0.2821 - val_loss: nan - val_accuracy: 0.2812
Epoch 4/25
33/33 [==============================] - 5s 147ms/step - loss: nan - accuracy: 0.2802 - val_loss: nan - val_accuracy: 0.2734
Epoch 5/25
33/33 [==============================] - 5s 148ms/step - loss: nan - accuracy: 0.2821 - val_loss: nan - val_accuracy: 0.2891
Epoch 6/25
23/33 [===================>..........] - ETA: 1s - loss: nan - accuracy: 0.3047

KeyboardInterrupt: 

In [ ]:
#Salva o modelo (pesos + conexões entre os neurônios, ou seja, a estrutura da rede)
model.save('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_simples' + ".model");